In [88]:
import json
import os
import pandas as pd
from tqdm import tqdm

In [22]:
for path in ['negeri', 'daerah', 'mukim']:
    files = os.listdir(path=path)
    files = [os.path.join(path, f) for f in files]
    files

    df = pd.DataFrame()
    for file_path in files:
        with open(file=file_path) as f:
            data = json.load(f)

        if 'error' in data:
            print(f'No data in {file_path}')
            continue

        feature = data.get('feature', {}).get('attributes', {})
        df = pd.concat([df, pd.DataFrame([feature])])

    df.reset_index(drop=True, inplace=True)
    df.to_csv(f'{path}_data_directory.csv', index=False)

No data in daerah\11.json


In [32]:
negeri_df = pd.read_csv('negeri_data_directory.csv')
daerah_df = pd.read_csv('daerah_data_directory.csv')
mukim_df = pd.read_csv('mukim_data_directory.csv')

negeri_df.rename(columns={'nama_neger': 'NEGERI'}, inplace=True)
daerah_df.rename(columns={'nama_daera': 'DAERAH'}, inplace=True)
mukim_df.rename(columns={'nama_mukim': 'MUKIM'}, inplace=True)

daerah_df = pd.merge(daerah_df, negeri_df[['kod_negeri', 'NEGERI']], on='kod_negeri', how='left')
mukim_df = pd.merge(mukim_df, daerah_df[['kod_daerah', 'DAERAH', 'NEGERI']], on='kod_daerah', how='left')

display(negeri_df.head())
display(daerah_df.head())
display(mukim_df.head())

,OBJECTID,kod_negeri,NEGERI
0,1,1,Johor
1,10,10,Selangor
2,11,11,Terengganu
3,12,14,W.P. Kuala Lumpur
4,13,15,W.P. Labuan


,OBJECTID,kod_negeri,kod_daerah,DAERAH,NEGERI
0,1,1,101,Batu Pahat,Johor
1,10,2,201,Kota Setar,Kedah
2,12,2,203,Padang Terap,Kedah
3,13,2,204,Langkawi,Kedah
4,14,2,205,Kuala Muda,Kedah


,OBJECTID,kod_negeri,kod_daerah,kod_mukim,MUKIM,DAERAH,NEGERI
0,1,1,101,10101,Bagan,Batu Pahat,Johor
1,10,1,101,10110,Simpang Kanan,Batu Pahat,Johor
2,100,1,108,10810,Sermin,Segamat,Johor
3,1000,6,606,60607,Pekan,Pekan,Pahang
4,1001,6,606,60608,Penyor,Pekan,Pahang


In [33]:
negeri_df.to_csv('negeri_data_directory.csv', index=False)
daerah_df.to_csv('daerah_data_directory.csv', index=False)
mukim_df.to_csv('mukim_data_directory.csv', index=False)

negeri_df = pd.read_csv('negeri_data_directory.csv')
daerah_df = pd.read_csv('daerah_data_directory.csv')
mukim_df = pd.read_csv('mukim_data_directory.csv')

display(negeri_df.head())
display(daerah_df.head())
display(mukim_df.head())

,OBJECTID,kod_negeri,NEGERI
0,1,1,Johor
1,10,10,Selangor
2,11,11,Terengganu
3,12,14,W.P. Kuala Lumpur
4,13,15,W.P. Labuan


,OBJECTID,kod_negeri,kod_daerah,DAERAH,NEGERI
0,1,1,101,Batu Pahat,Johor
1,10,2,201,Kota Setar,Kedah
2,12,2,203,Padang Terap,Kedah
3,13,2,204,Langkawi,Kedah
4,14,2,205,Kuala Muda,Kedah


,OBJECTID,kod_negeri,kod_daerah,kod_mukim,MUKIM,DAERAH,NEGERI
0,1,1,101,10101,Bagan,Batu Pahat,Johor
1,10,1,101,10110,Simpang Kanan,Batu Pahat,Johor
2,100,1,108,10810,Sermin,Segamat,Johor
3,1000,6,606,60607,Pekan,Pekan,Pahang
4,1001,6,606,60608,Penyor,Pekan,Pahang


In [90]:

for path in ['negeri', 'daerah', 'mukim']:
    files = os.listdir(path=path)
    files = [os.path.join(path, f) for f in files]
    df = pd.read_csv(f'{path}_data_directory.csv')

    for i in tqdm(range(len(files)), desc=f'Processing {path}...'):
        file_path = files[i]
        with open(file=file_path) as f:
            data = json.load(f)

        if 'error' in data:
            print(f'Error in file {file_path}.')
            continue
        
        kods = []
        if path=='negeri' and 'nama_neger' in data['feature']['attributes']:
            false_key = 'nama_neger'
            data['feature']['attributes'][path.upper()] = data['feature']['attributes'][false_key]
            del data['feature']['attributes'][false_key]

            del data['feature']['attributes']['kod_negeri']

            # print(data['feature']['attributes'])

        elif path=='daerah' and 'nama_daera' in data['feature']['attributes']:
            false_key = 'nama_daera'
            data['feature']['attributes'][path.upper()] = data['feature']['attributes'][false_key]
            del data['feature']['attributes'][false_key]

            filter_df = df[df[f'kod_{path}']==int(data['feature']['attributes'][f'kod_{path}'])]
            data['feature']['attributes']['NEGERI'] = filter_df['NEGERI'].to_list()[0]

            del data['feature']['attributes']['kod_negeri']
            del data['feature']['attributes']['kod_daerah']

            # display(filter_df)
            # print(data['feature']['attributes'])

        elif path=='mukim' and 'nama_mukim' in data['feature']['attributes']:
            false_key = 'nama_mukim'
            data['feature']['attributes'][path.upper()] = data['feature']['attributes'][false_key]
            del data['feature']['attributes'][false_key]

            filter_df = df[df[f'kod_{path}']==int(data['feature']['attributes'][f'kod_{path}'])]
            data['feature']['attributes']['DAERAH'] = filter_df['DAERAH'].to_list()[0]
            data['feature']['attributes']['NEGERI'] = filter_df['NEGERI'].to_list()[0]

            del data['feature']['attributes']['kod_negeri']
            del data['feature']['attributes']['kod_daerah']
            del data['feature']['attributes']['kod_mukim']

            # display(filter_df)
            # print(data['feature']['attributes'])

        json_object = json.dumps(data, indent=4)
        with open(f"processed_data/{file_path}", "w") as outfile:
            outfile.write(json_object)

        

Processing daerah...:   0%|          | 0/94 [00:00<?, ?it/s]

Error in file daerah\11.json.


Processing mukim...: 100%|██████████| 1730/1730 [00:09<00:00, 179.14it/s]


In [91]:
negeri_df = pd.read_csv('negeri_data_directory.csv')
daerah_df = pd.read_csv('daerah_data_directory.csv')
mukim_df = pd.read_csv('mukim_data_directory.csv')

In [ ]:
mukim_df[mukim_df['MUKIM']=='Seberang Perai']